### Initialization
$\rightarrow$ Setting up the libraries and initializing the sql connection.

In [12]:
import pandas as pd
from mysql import connector
import warnings
cnx = connector.connect(user="root", password="rootroot", host='127.0.0.1', database='sakila') # Connecting to Locally hosted MySQLDB
warnings.filterwarnings('ignore', category=UserWarning) # Suppressing warning that are not important

#### 1) Find the staff with the highest average sales per month for the last 10 months
$\rightarrow$ This query can be used to find out staff members who are consistently performing highly.

In [13]:
query = """
SELECT STAFF.STAFF_ID, AVG(PAYMENT.AMOUNT) AS 'AVERAGE SALES', DATE_FORMAT(PAYMENT.PAYMENT_DATE, '%M/%Y') AS MONTH
FROM SAKILA.STAFF
         INNER JOIN SAKILA.PAYMENT ON STAFF.STAFF_ID = PAYMENT.STAFF_ID
GROUP BY MONTH(PAYMENT.PAYMENT_DATE)
ORDER BY PAYMENT.PAYMENT_DATE DESC,2
LIMIT 10;
"""
query1 = pd.read_sql(query, cnx)
print(query1.to_string())

   STAFF_ID  AVERAGE SALES          MONTH
0         1       2.825165  February/2006
1         1       4.232835    August/2005
2         1       4.227968      July/2005
3         1       4.166038      June/2005
4         1       4.169775       May/2005


#### 2) Find the top 5 actors with highest amount of rentals, order the set by descending rental counts
$\rightarrow$ This query can be used to find if actors are in high demand.

In [14]:
query = """
SELECT CONCAT(A.FIRST_NAME, ' ', A.LAST_NAME) AS NAME, COUNT(R.RENTAL_ID) AS RENTAL_COUNT
FROM SAKILA.ACTOR AS A
         INNER JOIN SAKILA.FILM_ACTOR AS F ON A.ACTOR_ID = F.ACTOR_ID
         INNER JOIN SAKILA.INVENTORY AS I ON I.FILM_ID = F.FILM_ID
        INNER JOIN SAKILA.RENTAL AS R ON I.INVENTORY_ID = R.INVENTORY_ID
GROUP BY NAME
ORDER BY RENTAL_COUNT DESC
LIMIT 5;
"""
query2 = pd.read_sql(query, cnx)
print(query2.to_string())

                 NAME  RENTAL_COUNT
0         SUSAN DAVIS           825
1      GINA DEGENERES           753
2      MATTHEW CARREY           678
3         MARY KEITEL           674
4  ANGELA WITHERSPOON           654


#### 3) Find the most rented movie, and find any stores that do not have it in their inventory.
$\rightarrow$ This query can be used to find the most used payment type and depending on costs, provide benefits for under used payment type.

In [15]:
query = """
SELECT S.STORE_ID
FROM SAKILA.STORE AS S
         INNER JOIN SAKILA.INVENTORY AS I ON S.STORE_ID = I.STORE_ID
WHERE (SELECT F.FILM_ID
       FROM SAKILA.INVENTORY AS F
                INNER JOIN SAKILA.RENTAL AS R ON F.INVENTORY_ID = R.INVENTORY_ID
       GROUP BY F.FILM_ID
       LIMIT 1) NOT IN (SELECT S1.FILM_ID FROM SAKILA.INVENTORY AS S1 WHERE S1.STORE_ID = S.STORE_ID)
GROUP BY S.STORE_ID;
"""
query3 = pd.read_sql(query, cnx)
print(query3.to_string())

Empty DataFrame
Columns: [STORE_ID]
Index: []


#### 4) Find the top 5 cities with the most number of rentals.
$\rightarrow$ This query can be used to find the most popular cities.

In [16]:
query = """
SELECT CI.CITY, COUNT(R.RENTAL_ID)
FROM SAKILA.RENTAL AS R
         INNER JOIN SAKILA.CUSTOMER AS C ON R.CUSTOMER_ID = C.CUSTOMER_ID
         INNER JOIN SAKILA.ADDRESS AS A ON C.ADDRESS_ID = A.ADDRESS_ID
         INNER JOIN SAKILA.CITY AS CI ON A.CITY_ID = CI.CITY_ID
GROUP BY CI.CITY
ORDER BY 2 DESC
LIMIT 5;
"""
query4 = pd.read_sql(query, cnx)
print(query4.to_string())

          CITY  COUNT(R.RENTAL_ID)
0       Aurora                  50
1       London                  48
2  Saint-Denis                  46
3   Cape Coral                  45
4    Molodetno                  42


#### 5) Find the top 5 most popular stores and display their formatted address, ordered by most performing.
$\rightarrow$ This query can be used to find the most popular Stores.

In [17]:
query = """
SELECT S.STORE_ID,
       (SELECT CONCAT(A.ADDRESS, ' ', A.POSTAL_CODE, ',', (SELECT CONCAT(C.CITY, ',', (SELECT CO.COUNTRY
                                                                                       FROM SAKILA.COUNTRY AS CO
                                                                                       WHERE CO.COUNTRY_ID = C.COUNTRY_ID))
                                                           FROM SAKILA.CITY AS C
                                                           WHERE C.CITY_ID = A.CITY_ID))
        FROM SAKILA.ADDRESS AS A
        WHERE A.ADDRESS_ID = S.ADDRESS_ID) AS ADDRESS,
       COUNT(S.STORE_ID)                   AS RENTAL_COUNT
FROM SAKILA.RENTAL AS R
         INNER JOIN SAKILA.INVENTORY AS I ON R.INVENTORY_ID = I.INVENTORY_ID
         INNER JOIN SAKILA.STORE AS S ON I.STORE_ID = S.STORE_ID
GROUP BY S.STORE_ID
ORDER BY RENTAL_COUNT DESC
LIMIT 5;
"""
query4 = pd.read_sql(query, cnx)
print(query4.to_string())

   STORE_ID                                  ADDRESS  RENTAL_COUNT
0         2  28 MySQL Boulevard ,Woodridge,Australia          8121
1         1     47 MySakila Drive ,Lethbridge,Canada          7923
